***
***
> >>>>>>> # ***parallelize larch***
***
***

In [65]:
%matplotlib qt
from larch.xafs import preedge
from larch.math.fitpeak import fit_peak
import h5py, os, time, timeit, numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt

[numexpr.utils] INFO : Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[numexpr.utils] INFO : NumExpr defaulting to 8 threads.
[dxchange.reader] WARNING : astropy module not found


In [66]:
fn = '/your_dir/test.h5'
with h5py.File(fn, 'r') as f:
    data = f['/spec'][:]
    eng = f['/eng'][:]

dim = data.shape
print(dim)

### pre_edge
# ids = list(np.arange(20)) + list(np.arange(40, 49))
### fit_peak
ids = list(np.arange(40, 54))
x = eng[ids]
y = data[ids, :].reshape([len(ids), -1])

n_cpu = os.cpu_count()
par_dict = {'amplitude':1e-3, 'center':x[-1], 'sigma':3.0,
            'slope':0.1, 'intercept':0.0}
par_bounds = ((1e-5, x[-8], 0.1, 0, 0), (1e-2, x[-1], 10, 1, 2))

print(x.shape, y.shape)
print(time.asctime())    
############ multiprocessing
with mp.Pool(n_cpu-1) as pool:
    # rlt = pool.starmap(preedge, [(x, y[:, ii]) for ii in np.int32(np.arange(dim[1]*dim[2]))])
    rlt = pool.starmap(fit_peak, [(x, y[:, ii], 'lorentzian') for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.join()
pool.close()

print(time.asctime())
#print(len(rlt))

(101, 150, 130)
(14,) (14, 19500)
Tue Jun 16 10:35:24 2020
Tue Jun 16 10:37:59 2020


In [8]:
print((rlt[0].fit))

[-6.5672310e-07 -7.5151883e-07 -8.6970380e-07 -1.0166663e-06
 -1.2063750e-06 -1.4518193e-06 -1.7843104e-06 -2.2402783e-06
 -2.9040248e-06 -3.9033016e-06 -5.5066685e-06 -8.4416870e-06
 -1.4391217e-05 -2.9843643e-05]


In [10]:
print(rlt[0]._members().keys())

dict_keys(['x', 'y', 'dy', 'model', 'background', 'form', 'init_params', 'fit', 'fit_init', 'fit_details', 'chi_square', 'chi_reduced', 'aic', 'bic', 'covar', 'rfactor', 'params', 'nvarys', 'nfree', 'ndata', 'var_names', 'nfev', 'success', 'errorbars', 'message', 'lmdif_message', 'residual'])


In [25]:
plt.figure(1)
for ii in range(50, 55):
    for jj in range(75, 80):
        idx = ii*dim[2] + jj
        plt.plot(rlt[0].x, rlt[idx].y)
        #plt.plot(rlt[0].x, rlt[idx].fit)

In [20]:
plt.figure(2)
plt.imshow(y[10].reshape([dim[1], dim[2]]))

In [56]:
a = data
b = np.reshape(a, (a.shape[0], -1))
print(a.shape, b.shape)

(101, 150, 130) (101, 19500)


***
***
> >>>>>>> # ***parallelize scipy.optimize.curve_fit***
***
***

In [1]:
import numpy as np
from scipy.optimize import minimize, curve_fit, least_squares as lsq, lsq_linear, nnls
import multiprocess as mp

from lmfit.lineshapes import (gaussian, lorentzian, voigt, pvoigt, moffat, pearson7,
                         breit_wigner, damped_oscillator, dho, logistic, lognormal,
                         students_t, expgaussian, donaich, skewed_gaussian,
                         skewed_voigt, step, rectangle,  
                         exponential, powerlaw, linear, parabolic, sine, 
                         expsine, split_lorentzian)

functions = {'gaussian':gaussian, 'lorentzian':lorentzian, 'voigt':voigt, 
             'pvoigt':pvoigt, 'moffat':moffat, 'pearson7':pearson7,
             'breit_wigner':breit_wigner, 'damped_oscillator':damped_oscillator,
             'dho':dho, 'logistic':logistic, 'lognormal':lognormal,
             'students_t':students_t, 'expgaussian':expgaussian, 
             'donaich':donaich, 'skewed_gaussian':skewed_gaussian,
             'skewed_voigt':skewed_voigt, 'step':step, 'rectangle':rectangle, 
             'exponential':exponential, 'powerlaw':powerlaw, 
             'linear':linear, 'parabolic':parabolic, 'sine':sine, 
             'expsine':expsine, 'split_lorentzian':split_lorentzian}

In [51]:
import h5py, os, time
fn = '/your_dir/test.h5'
with h5py.File(fn, 'r') as f:
    data = f['/spec'][:]
    eng = f['/eng'][:]

ids = list(np.arange(40, 54))
x = eng[ids]
y = data[ids, :].reshape([len(ids), -1])

func = functions['lorentzian']   
dim = data.shape
print(dim)
print(x.shape, y.shape)

def res(x0, y0=None):
    return func(x0) - y0
    #rlt_t = lsq(func(x0)-y0, x0, bounds=(x0[0], x0[-1]))
    #return rlt_t
def local_lsq(x0, y0):
    lsq(res, x0, **{y0:y0})

n_cpu = os.cpu_count()
pars_ini = {}
#bnds = (x0[0], x0[-1])
print(time.asctime())
############ multiprocessing
with mp.Pool(n_cpu-1) as pool:
    rlt = pool.starmap(local_lsq, [(x, y[:, ii]) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.join()
pool.close()
print(time.asctime())

(101, 150, 130)
(14,) (14, 19500)
Tue Jun 16 10:03:36 2020


TypeError: unhashable type: 'numpy.ndarray'

In [25]:
"""
The res definition here works but it IS WRONG in meanings. func is a Lorentzian lineshape defined in lmfit. Its first argument is x, and Lorentzian
parmeters are defined as keywords with default values. The definition below assume the Lorentzian function has fixed parameters defined as their
default values and fit x to minimize the the fitted value and the given value.

To use lsq for curve fitting, we need to define a loss function in form of loss(func, fvars, x, y): return sum((func(fvars, x) - y)**2)
"""
def res(x0, *args):
    return func(x0) - args[0]

def local_lsq(x0, y0):
    lsq(res, x0, args=(y0))
    
res(x, y[:, 0])
r = lsq(res, (1, x[10], 2), args=(y[:, 0]))

***
***
> >>>>>>> # ***parallelize scipy.optimize.curve_fit***
***
***

In [ ]:
"""
use scipy.optimize.curve_fit to fit experimental data
The easiest way to do curve fitting is to use curve_fit. The loss function is defined internally.
However, the convergence tolerance cannot be changed in curve_fit. The below case failed for exceeding
maximum allowed iterations.
"""
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, curve_fit, least_squares as lsq, lsq_linear, nnls
import multiprocess as mp
import h5py, os, time

from lmfit.lineshapes import (gaussian, lorentzian, voigt, pvoigt, moffat, pearson7,
                         breit_wigner, damped_oscillator, dho, logistic, lognormal,
                         students_t, expgaussian, donaich, skewed_gaussian,
                         skewed_voigt, step, rectangle,  
                         exponential, powerlaw, linear, parabolic, sine, 
                         expsine, split_lorentzian)

functions = {'gaussian':gaussian, 'lorentzian':lorentzian, 'voigt':voigt, 
             'pvoigt':pvoigt, 'moffat':moffat, 'pearson7':pearson7,
             'breit_wigner':breit_wigner, 'damped_oscillator':damped_oscillator,
             'dho':dho, 'logistic':logistic, 'lognormal':lognormal,
             'students_t':students_t, 'expgaussian':expgaussian, 
             'donaich':donaich, 'skewed_gaussian':skewed_gaussian,
             'skewed_voigt':skewed_voigt, 'step':step, 'rectangle':rectangle, 
             'exponential':exponential, 'powerlaw':powerlaw, 
             'linear':linear, 'parabolic':parabolic, 'sine':sine, 
             'expsine':expsine, 'split_lorentzian':split_lorentzian}

In [31]:
import h5py, os, time
fn = '/your_dir/test.h5'
with h5py.File(fn, 'r') as f:
    data = f['/spec'][:]
    eng = f['/eng'][:]

ids = list(np.arange(40, 54))
x = eng[ids]
y = data[ids, :].reshape([len(ids), -1])

func = functions['lorentzian']   
dim = data.shape
print(dim)
print(x.shape, y.shape)

def fitting(func, x0, y0, xinits, bnds=None):
    curve_fit(func, x0, y0, p0=xinits, bounds=bnds)

print(time.asctime())

bnds = ((0.01, x[0], 0.1), (5, x[-1], 13))
n_cpu = os.cpu_count()
with mp.Pool(n_cpu - 1) as pool:
    rlt = pool.starmap(fitting, [(func, x, y[:,ii], (1, x[10], 2), bnds) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.close()
pool.join()

print(time.asctime())

(101, 383, 1040)
(14,) (14, 398320)
Mon Jun 15 10:52:49 2020


RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

***
***
# ***parallelize a customized curve fitting function with scipy.optimize.least_squares***
***
***

In [1]:
"""
use scipy.optimize.curve_fit to fit experimental data
The easiest way to do curve fitting is to use curve_fit. The loss function is defined internally.
However, the convergence tolerance cannot be changed in curve_fit. The below case failed for exceeding
maximum allowed iterations.
"""
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, curve_fit, least_squares as lsq, lsq_linear, nnls
import multiprocess as mp
import h5py, os, time

from lmfit.lineshapes import (gaussian, lorentzian, voigt, pvoigt, moffat, pearson7,
                         breit_wigner, damped_oscillator, dho, logistic, lognormal,
                         students_t, expgaussian, donaich, skewed_gaussian,
                         skewed_voigt, step, rectangle,  
                         exponential, powerlaw, linear, parabolic, sine, 
                         expsine, split_lorentzian)

functions = {'gaussian':gaussian, 'lorentzian':lorentzian, 'voigt':voigt, 
             'pvoigt':pvoigt, 'moffat':moffat, 'pearson7':pearson7,
             'breit_wigner':breit_wigner, 'damped_oscillator':damped_oscillator,
             'dho':dho, 'logistic':logistic, 'lognormal':lognormal,
             'students_t':students_t, 'expgaussian':expgaussian, 
             'donaich':donaich, 'skewed_gaussian':skewed_gaussian,
             'skewed_voigt':skewed_voigt, 'step':step, 'rectangle':rectangle, 
             'exponential':exponential, 'powerlaw':powerlaw, 
             'linear':linear, 'parabolic':parabolic, 'sine':sine, 
             'expsine':expsine, 'split_lorentzian':split_lorentzian}

In [2]:
fn = '/media/xiao_usb/test_data_large.h5'
with h5py.File(fn, 'r') as f:
    data = f['/spec'][:]
    eng = f['/eng'][:]

ids = list(np.arange(40, 54))
x = eng[ids]
y = data[ids, :].reshape([len(ids), -1])

func = functions['lorentzian']   
dim = data.shape
print(dim)
print(x.shape, y.shape)

def f(fvars, func, x, y):
    return (func(x, fvars[0], fvars[1], fvars[2]) - y)

def my_lsq(f, fvars, func, x, y, bnds, ftol, xtol):
    #return lsq(f, fvars, bounds=bnds, ftol=ftol, xtol=xtol, args=(func, x, y))
    return lsq(f, fvars, ftol=ftol, xtol=xtol, args=(func, x, y))



(101, 333, 330)
(14,) (14, 109890)


In [3]:
print(time.asctime())

bnds = ([0.01, x[0], 0.1], [5, x[-1]+10, 13])
ftol = 1e-7
xtol = 1e-7
n_cpu = os.cpu_count()
with mp.Pool(n_cpu - 1) as pool:
    rlt = pool.starmap(my_lsq, [(f, (1, x[10], 2), func, x, y[:, ii], bnds, ftol, xtol) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.close()
pool.join()

print(time.asctime())

Sun Jun 28 07:45:36 2020
Sun Jun 28 07:45:56 2020


In [34]:
bnds = ([0.01, x[0], 0.1], [5, x[-1], 13])
ftol = 1e-7
xtol = 1e-7
gtol = 1e-16
tiny = 1e-16

#def f(fvars, x, y):
#    return ((fvars[0]/(1 + ((1.0*x-fvars[1])/max(tiny, fvars[2]))**2))
#            / max(tiny, (np.pi*fvars[2]))) - y
#def my_lsq(f, fvars, x, y, jac, bnds, method, ftol, xtol, gtol):
#    return lsq(f, fvars, jac=jac, bounds=bnds, method=method, ftol=ftol, xtol=xtol, gtol=gtol, args=(x, y))
#bnds = (-np.inf, np.inf)
#ii = 1000
#r1 = my_lsq(f, (0.1, 8352, 0.1), x, y[:, ii], '3-point', bnds, 'trf', ftol, xtol, gtol)
#fr = f((0.1, x[10], 0.1), x, y[:, ii])

#def f(fvars, func, x, y):
def f(fvars, x, y):
    return (func(x, fvars[0], fvars[1], fvars[2]) - y)
def my_lsq(f, fvars, x, y, jac, bnds, method, ftol, xtol, gtol):
    return lsq(f, fvars, jac=jac, bounds=bnds, method=method, ftol=ftol, xtol=xtol, gtol=gtol, args=(x, y))

#def my_lsq(f, fvars, func, x, y, bnds, ftol, xtol, gtol):
#    return lsq(f, fvars, bounds=bnds, ftol=ftol, xtol=xtol, gtol=gtol, args=(func, x, y))

bnds = (-np.inf, np.inf)
ii = 1000
#r1 = my_lsq(f, (0.1, x[10], 0.1), func, x, y[:, ii], bnds, ftol, xtol, gtol)
#fr = f((0.1, x[10], 0.1), func, x, y[:, ii])
r1 = my_lsq(f, (0.1, 8352, 0.1), x, y[:, ii], '3-point', bnds, 'trf', ftol, xtol, gtol)
fr = f((0.1, x[10], 0.1), x, y[:, ii])
#func(x, 1, x[10], 2)
print(r1)
print(fr)
print(y[:, ii])

 active_mask: array([0., 0., 0.])
        cost: 9.683704376220703
         fun: array([-1.0138135 ,  0.30978107, -0.6724484 ,  0.96750426,  0.43831325,
        1.0598409 , -2.1402411 ,  0.59362984,  0.9861753 , -0.648047  ,
       -2.076005  ,  1.7875478 ,  1.0370758 , -0.79612136], dtype=float32)
        grad: array([-0.00033019, -0.00170643,  0.00813498])
         jac: array([[ 0.02321326, -0.35396862, -0.02673043],
       [ 0.02655456, -0.39479682, -0.09355652],
       [ 0.03007172, -0.42324099, -0.18176694],
       [ 0.03411646, -0.42524761, -0.29136172],
       [ 0.03798534, -0.38482025, -0.42234084],
       [ 0.04044735, -0.29112878, -0.54530084],
       [ 0.04255765, -0.14454339, -0.63083822],
       [ 0.04290936,  0.0300688 , -0.64687645],
       [ 0.04220593,  0.19988489, -0.60410774],
       [ 0.04009563,  0.32919109, -0.50253212],
       [ 0.03657847,  0.40381604, -0.37957212],
       [ 0.03270959,  0.42848274, -0.24859302],
       [ 0.02919243,  0.41573793, -0.15236346],
  

In [37]:
plt.figure(1)
plt.plot(x, func(x, r1.x[0], r1.x[1], r1.x[2]))
plt.plot(x, y[:, ii])

In [53]:
def f(fvars, x, y):
    return (func(x, fvars[0], fvars[1], fvars[2]) - y)
def my_lsq(f, fvars, x, y, jac, bnds, method, ftol, xtol, gtol):
    return lsq(f, fvars, jac=jac, bounds=bnds, method=method, ftol=ftol, xtol=xtol, gtol=gtol, args=(x, y))

In [55]:
ftol = 1e-7
xtol = 1e-7
gtol = 1e-7
tiny = 1e-16
jac = '3-point'
method = 'trf'
bnds = ((-np.inf, x[0], 0.01), (np.inf, x[-1], 100))

print(time.asctime())

n_cpu = os.cpu_count()
with mp.Pool(n_cpu - 1) as pool:
    rlt = pool.starmap(my_lsq, [(f, (1, x[10], 2), x, y[:, ii], jac, bnds, method, ftol, xtol, gtol) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.close()
pool.join()

print(time.asctime())

Tue Jun 16 10:06:13 2020
Tue Jun 16 10:06:24 2020


In [42]:
print(len(rlt))
print(data.shape)

398320
(101, 383, 1040)


In [56]:
plt.figure(2)
plt.imshow(data[50, :])

In [61]:
plt.figure(3)
#idx = 106*dim[2]+500
#plt.plot(x, func(x, rlt[idx].x[0], rlt[idx].x[1], rlt[idx].x[2]))
#plt.plot(x, y[:, idx])
xd = np.linspace(x[0], x[-1], x.shape[0]*100)
for ii in range(55, 65):
    for jj in range(70, 80):
        idx = ii*dim[2]+jj
        plt.plot(xd, func(xd, rlt[idx].x[0], rlt[idx].x[1], rlt[idx].x[2]))
        plt.plot(x, y[:, idx])

>>> ## ***generalize the peak fitting function***

In [62]:
fn = '/your_dir/test.h5'
with h5py.File(fn, 'r') as f:
    data = f['/spec'][:]
    eng = f['/eng'][:]

ids = list(np.arange(40, 54))
x = eng[ids]
y = data[ids, :].reshape([len(ids), -1])

def f(fvars, func, x, y):
    return (func(x, fvars[0], fvars[1], fvars[2]) - y)
def my_lsq(f, fvars, func, x, y, jac, bnds, method, ftol, xtol, gtol):
    return lsq(f, fvars, jac=jac, bounds=bnds, method=method, ftol=ftol, xtol=xtol, gtol=gtol, args=(func, x, y))

In [63]:
ftol = 1e-7
xtol = 1e-7
gtol = 1e-7
tiny = 1e-16
jac = '3-point'
method = 'trf'
bnds = ((-np.inf, x[0], 0.01), (np.inf, x[-1], 100))

print(time.asctime())

n_cpu = os.cpu_count()
with mp.Pool(n_cpu - 1) as pool:
    rlt = pool.starmap(my_lsq, [(f, (1, x[10], 2), func, x, y[:, ii], jac, bnds, method, ftol, xtol, gtol) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.close()
pool.join()

print(time.asctime())

Tue Jun 16 10:27:48 2020
Tue Jun 16 10:27:58 2020
